<a href="https://colab.research.google.com/github/derrodo61/notebooks/blob/main/neo4j_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create using merge

## Create nodes

```
LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/titles_and_skills.csv'
AS row
MERGE (t:Title {name: row.Title})
```

```
LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/skills_details.csv'
AS row
MERGE (s:Skill {name: row.Skill})
SET
s.id = row.ID,
s.description = row.Description,
s.category=row.Category
```

## Add property

```
LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/titles_and_skills.csv'
AS row
MATCH (t:Title {name: row.Title})
SET t.skills = split(row.Skills, '|')
```

## Remove property

```
MATCH (t:Title) REMOVE t.skills
```

## Create edges / relations

```
MATCH (t:Title)
UNWIND t.skills AS skill
MERGE (s:Skill {name: skill})
MERGE (t)-[:REQUIRES]->(s)
```

## Add additional labels

```
MATCH (s:Skill{category:'Certification'})
SET s:Skill:Certification
RETURN s.name, labels(s)

MATCH (s:Skill{category:'SpecializedSkill'})
SET s:Skill:SpecializedSkill
RETURN s.name, labels(s)

MATCH (s:Skill) REMOVE s.category
```

## Match

```
match (n) RETURN n
```

```
MATCH (n:Skill|Title) RETURN n
```


```
MATCH (n:Title) RETURN n LIMIT 25
```


```
MATCH (t:Title) RETURN t.name AS name, t.skills AS skills LIMIT 5
```


```
MATCH (s:Skill) RETURN DISTINCT s.category
```

## Schema procedures

```
call db.schema.visualization
call db.schema.nodeTypeProperties
call db.schema.relTypeProperties
```